In [1]:
# data reading, processing before join tables

# imports
import sys
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pdb
import pandas as pd
from functools import reduce as rd
from sklearn.decomposition import PCA
%reload_ext autoreload
%autoreload 2

from pyspark.sql.functions import avg, min, max

In [2]:
# set up constants for ease of directory use
FINAL_DIR = 'gs://final-bucket-jy'
DATA_DIR = f'{FINAL_DIR}/data/'

In [3]:
# start a spark session
from pyspark.sql import SparkSession

app_name = "final_project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .config('spark.executor.memory', '4g')\
        .config("spark.sql.debug.maxToStringFields", 100)\
        .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/14 18:50:46 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
24/04/14 18:50:46 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
24/04/14 18:50:46 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/14 18:50:46 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [4]:
#sc.stop()

In [5]:
# read from google cloud using spark

fdwind = spark.read.csv(DATA_DIR + 'training2_fdwind.csv',inferSchema=True,header=True)
fdwindairport = spark.read.csv(DATA_DIR + 'training2_fdwindairport.csv',inferSchema=True,header=True)
fdwindaltitude = spark.read.csv(DATA_DIR + 'training2_fdwindaltitude.csv',inferSchema=True,header=True)
flighthistory = spark.read.csv(DATA_DIR + 'training2_flighthistory.csv',inferSchema=True,header=True)
asdiflightplan = spark.read.csv(DATA_DIR + 'training2_asdiflightplan.csv',inferSchema=True,header=True)

# 0. Additional feature processing ideas

<div style="background-color:#f0f0f0; padding:10px;">
<strong>Summary-fdwind:</strong><br>
There are no NULL values in the columns 'fbwindairportid', 'ordinal', 'bearing', and 'knots'. Missing values exist only in the 'temperature' column, which may result from issues in observation recording. Knowing the mean of all temperature values, we have decided to replace the NULL values with this mean for further analysis.
</div>

<div style="background-color:#f0f0f0; padding:10px;">
<strong>Summary-fdwindairport:</strong><br>
The 'fdwindairport' table has 3 columns, and no missing values were detected, which is great!
</div>

<div style="background-color:#f0f0f0; padding:10px;">
<strong>Summary-fdwindaltitude:</strong><br>
The 'fdwindaltitude' table has 3 columns, and no missing values were detected, which is great!
</div>

<div style="background-color:#f0f0f0; padding:10px;">
<strong>Summary-flighthistory:</strong><br>
The 'flighthistory' table includes various columns related to flight information, such as identifiers, codes, and timestamps, among others. Missing values are present in the columns 'airline_icao_code', 'published_departure', 'published_arrival', 'scheduled_runway_arrival', 'actual_runway_arrival', 'scheduled_air_time', 'scheduled_block_time', 'scheduled_aircraft_type', 'actual_aircraft_type', and 'icao_aircraft_type_actual'. 
    
After reviewing these missing values, we came up a detailed way to handle those Null values (in Milestone 1).   
1. For columns that are missing all:   
"actual_aircraft_type": the "icao_aircraft_type_actual" can be used to represent the same information  
<font color=blue>SOLUTION: detele the whole column, "actual_aircraft_type" </font>  
2. For columns that are missing some:  
a. IMPORTANT column: "published_arrival"  
    Since we need "published_arrival" to decide whether the flight is delayed or not, it is important.  
    <font color=blue>SOLUTION: for those rows that has "published_arrival" missing => drop them </font>   
b. Not that important columns:   
<font color=blue>SOLUTION: We will keep those columns in the way that they originaly are so far </font>   
"airline_icao_code": not important since airline_code has no missing  
"published_departure": not important since we only need arrival time to decide whether the flight is delayed or not   
scheduled_gate_departure, actual_gate_departure, scheduled_gate_arrival, actual_gate_arrival,   
    scheduled_runway_departure, actual_runway_departure, scheduled_runway_arrival, actual_runway_arrival  
    scheduled_air_time, scheduled_block_time
"scheduled_aircraft_type": not too important  
"icao_aircraft_type_actual": not too important  

*For "scheduled_air_time", the min is negative, we want to double check:   
Base on its defination: Computed number of minutes from the scheduled runway departure to the scheduled runway arrival. it should be all positive.   
(Since all the time that showed in the data is in Pacific Time Zone, we do not need consider time difference.)


</div>

<div style="background-color:#f0f0f0; padding:10px;">
<strong>Summary-asdiflightplan:</strong><br>
The 'asdiflightplan' table contains records for 7,954,958 flights, as indicated by the counts for 'id', 'update_time_utc', 'flighthistory_id', among other columns. Missing values are present in the 'departure_airport' and 'arrival_airport' columns. As these missing values cannot be remedied, we have decided to remove the rows containing these NAs.
</div>

# 1. Join tables

## 1.1. fdwind_all.csv

1. outer join ( training2_fdwindaltitude.csv & training2_fdwindairport.csv ) with column “fbwindreportid”
2. outer join ( 1 & training2_fdwind.csv ) with column “fbwindairportid”, “ordinal”

The number of Columns after processing: 6

In [6]:
# 1. outer join ( training2_fdwindaltitude.csv & training2_fdwindairport.csv ) with column “fbwindreportid”

fdwind_al_ap = fdwindaltitude.join(fdwindairport, 'fbwindreportid', 'outer')

In [7]:
# 2. outer join ( 1 & training2_fdwind.csv ) with column “fbwindairportid”, “ordinal”
# AND drop two id columns that will not be used later

fdwind_all = fdwind_al_ap.join(fdwind, ['fbwindairportid', 'ordinal'], 'outer')\
                            .drop('fbwindairportid','fbwindreportid')

In [8]:
# Data processing: Aggregate 
# Steps:
# 1. group by airportcode
# 2. calculate average altitude, bearing, knots, temperature [ignore null value]

fdwind_all_agg = fdwind_all.groupBy('airportcode')\
                            .agg(avg('altitude').alias('avg_altitude'),\
                                 avg('bearing').alias('avg_bearing'),\
                                 avg('knots').alias('avg_knots'),\
                                 avg('temperature').alias('avg_temperature'),\
                                 min('altitude').alias('min_altitude'),\
                                 min('bearing').alias('min_bearing'),\
                                 min('knots').alias('min_knots'),\
                                 min('temperature').alias('min_temperature'),\
                                 max('altitude').alias('max_altitude'),\
                                 max('bearing').alias('max_bearing'),\
                                 max('knots').alias('max_knots'),\
                                 max('temperature').alias('max_temperature'))

In [9]:
# to cache fdwind_all_agg in rdd for later join

fdwind_all_agg_rdd = fdwind_all_agg.cache()

## 1.2. asdi_flighthistory.csv:

outer join ( training2_flighthistory.csv & training2_asdiflightplan.csv )
    with column “id” == “flighthistory_id”


In [10]:
# outer join ( training2_flighthistory.csv & training2_asdiflightplan.csv ) with column “flighthistory_id”

# Drop columns in flighthistory:
# 1. For useless columns: flighthistory_id, id, actual_aircraft_type(cuz missing all)
# 2. For columns that are duplicate: airline_icao_code, departure_airport_icao_code,
#           arrival_airport_icao_code,
# 3. Columns that are similar to published departure/arrival time:
#           scheduled_gate_departure, actual_gate_departure,
#           scheduled_gate_arrival, actual_gate_arrival,
#           scheduled_runway_departure, actual_runway_departure,
#           scheduled_runway_arrival, actual_runway_arrival,
# 4. Others not important columns:
#           departure_airport_timezone_offset, arrival_airport_timezone_offset,          

# Drop other columns in asdiflightplan:
# 1. useless: update_time_utc

asdi_flighthistory = flighthistory.withColumnRenamed('id','flighthistory_id').join(asdiflightplan, 'flighthistory_id', 'outer')\
                                    .drop('flighthistory_id', 'id', 
                                          'actual_aircraft_type',
                                          'airline_icao_code', 'departure_airport_icao_code', 'arrival_airport_icao_code',
                                          'scheduled_gate_departure', 'actual_gate_departure',
                                          'scheduled_gate_arrival', 'actual_gate_arrival',
                                          'scheduled_runway_departure', 'actual_runway_departure',
                                          'scheduled_runway_arrival', 'actual_runway_arrival',
                                          'departure_airport_timezone_offset', 'arrival_airport_timezone_offset',
                                          'update_time_utc',
                                          'estimated_departure_utc', 'estimated_arrival_utc', # those two are not for sure to drop
                                          'departure_airport', 'arrival_airport'
                                         )

In [11]:
# In case after we drop some column, there is duplicates in the data

asdi_fh_unique = asdi_flighthistory.dropDuplicates()

In [12]:
# cache as rdd

asdi_fh_unique_rdd = asdi_fh_unique.cache() # we should use data that has no duplicate 

## 1.3. fdwind_asdi_final.csv:

1. outer join ( fdwind_all.csv & asdi_flighthistory.csv ) with column “airportcode” == “arrival_airport_code”   
(remember to rename the columns in fdwind_all.csv to show it is for arrival airport)

2. outer join ( fdwind_all.csv & 1 ) with column “airportcode” == “departure_airport_code”   
(remember to rename the columns in fdwind_all.csv to show it is for departure airport)


In [13]:
# TODO-Join-Steps: This will help us get the final data
## For arrival airport:
# 1. fdwind: rename airportcode to arrival_airport_code
# 2. fdwind: rename all the rest column names with arrival in it
# 3. join with asdi_fh with arrival_airport_code
## For departure airport:
# 1. fdwind: rename airportcode to departure_airport_code
# 2. fdwind: rename all the rest column names with departure in it
# 3. join with asdi_fh with departure_airport_code


# 1. outer join ( fdwind_all.csv & asdi_flighthistory.csv ) with column “airportcode” == “arrival_airport_code”

fdwind_asdi_arrival = fdwind_all_agg_rdd\
    .withColumnRenamed('airportcode','arrival_airport_code')\
    .withColumnRenamed('avg_altitude','arrival_airport_avg_altitude')\
    .withColumnRenamed('avg_bearing','arrival_airport_avg_bearing')\
    .withColumnRenamed('avg_knots','arrival_airport_avg_knots')\
    .withColumnRenamed('avg_temperature','arrival_airport_avg_temperature')\
    .withColumnRenamed('min_altitude','arrival_airport_min_altitude')\
    .withColumnRenamed('min_bearing','arrival_airport_min_bearing')\
    .withColumnRenamed('min_knots','arrival_airport_min_knots')\
    .withColumnRenamed('min_temperature','arrival_airport_min_temperature')\
    .withColumnRenamed('max_altitude','arrival_airport_max_altitude')\
    .withColumnRenamed('max_bearing','arrival_airport_max_bearing')\
    .withColumnRenamed('max_knots','arrival_airport_max_knots')\
    .withColumnRenamed('max_temperature','arrival_airport_max_temperature')\
    .join(asdi_fh_unique_rdd, 
          'arrival_airport_code',
          'outer')

In [14]:
# 2. outer join ( fdwind_all.csv & 1 ) with column “airportcode” == “departure_airport_code”  

fdwind_asdi_final = fdwind_all_agg_rdd\
    .withColumnRenamed('airportcode','departure_airport_code')\
    .withColumnRenamed('avg_altitude','departure_airport_avg_altitude')\
    .withColumnRenamed('avg_bearing','departure_airport_avg_bearing')\
    .withColumnRenamed('avg_knots','departure_airport_avg_knots')\
    .withColumnRenamed('avg_temperature','departure_airport_avg_temperature')\
    .withColumnRenamed('min_altitude','departure_airport_min_altitude')\
    .withColumnRenamed('min_bearing','departure_airport_min_bearing')\
    .withColumnRenamed('min_knots','departure_airport_min_knots')\
    .withColumnRenamed('min_temperature','departure_airport_min_temperature')\
    .withColumnRenamed('max_altitude','departure_airport_max_altitude')\
    .withColumnRenamed('max_bearing','departure_airport_max_bearing')\
    .withColumnRenamed('max_knots','departure_airport_max_knots')\
    .withColumnRenamed('max_temperature','departure_airport_max_temperature')\
    .join(fdwind_asdi_arrival, 
          'departure_airport_code',
          'outer')

In [15]:
fdwind_asdi_final_rdd = fdwind_asdi_final.cache()

## 1.4. flight_full.csv

In this dataset, we only keep those records that are having both published_arrival and original_arrival_utc.   
*published_arrival: represent the official/actuall arrival time*  
*original_arrival_utc: represent the original time that the flight is been scheduled*  

The reason is that we want to use those two columns to help us decide whether the flight is delayed or not.  
Either of them missing will not helping us to label whether the flight is delayed so we would like to filter out:  
1. Rows that has "published_arrival" missing;  
2. Rows that has "original_arrival_utc" missing.

In [24]:
flights = fdwind_asdi_final_rdd.filter(fdwind_asdi_final_rdd['published_arrival'].isNotNull() & fdwind_asdi_final_rdd['original_arrival_utc'].isNotNull())

# 2. Data Engineering

## column name: delay

If original_arrival_utc < published_arrival, label it as 1 (delayed).  
Otherwise, label it as 0 (not delayed).

In [25]:
# Since the time data is in string type now, we want to convert it to timestamp
#      which is convinient for later labeling the data

from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import TimestampType

flights = flights.withColumn("published_departure_timestamp", to_timestamp(flights["published_departure"], "yyyy-MM-dd HH:mm:ss"))
flights = flights.withColumn("published_arrival_timestamp", to_timestamp(flights["published_arrival"], "yyyy-MM-dd HH:mm:ss"))
flights = flights.withColumn("original_departure_utc_timestamp", to_timestamp(flights["original_departure_utc"], "yyyy-MM-dd HH:mm:ss"))
flights = flights.withColumn("original_arrival_timestamp", to_timestamp(flights["original_arrival_utc"], "yyyy-MM-dd HH:mm:ss"))

In [26]:
flights = flights.drop("published_departure", "published_arrival", "original_departure_utc", "original_arrival_utc").cache()

24/04/14 18:53:02 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


In [27]:
flights.printSchema()

root
 |-- departure_airport_code: string (nullable = true)
 |-- departure_airport_avg_altitude: double (nullable = true)
 |-- departure_airport_avg_bearing: double (nullable = true)
 |-- departure_airport_avg_knots: double (nullable = true)
 |-- departure_airport_avg_temperature: double (nullable = true)
 |-- departure_airport_min_altitude: integer (nullable = true)
 |-- departure_airport_min_bearing: integer (nullable = true)
 |-- departure_airport_min_knots: integer (nullable = true)
 |-- departure_airport_min_temperature: integer (nullable = true)
 |-- departure_airport_max_altitude: integer (nullable = true)
 |-- departure_airport_max_bearing: integer (nullable = true)
 |-- departure_airport_max_knots: integer (nullable = true)
 |-- departure_airport_max_temperature: integer (nullable = true)
 |-- arrival_airport_code: string (nullable = true)
 |-- arrival_airport_avg_altitude: double (nullable = true)
 |-- arrival_airport_avg_bearing: double (nullable = true)
 |-- arrival_airport_

In [28]:
# For this and folloing chunk of code, we will:
# 1. create column to show the time differences between published arrival/departure & original arrival/departure
# 2. binary the time differences to delay (1) and not delay (0) in a 15 minutes grace period 

from pyspark.sql.functions import unix_timestamp, col, when

# Calculate the time difference in seconds
flight_label = flights.withColumn(
    "arrival_time_difference_seconds",
    (unix_timestamp("published_arrival_timestamp") - unix_timestamp("original_arrival_timestamp"))
)

##### TODO for next Milestone: figure out how to utilize the departure time #####
# So far this does not work because we did not filter out the rows that are missing published/original departure time
# flight_label = flight_label.withColumn(
#     "departure_time_difference_seconds",
#     (unix_timestamp("published_departure_timestamp") - unix_timestamp("original_departure_timestamp"))
# )

In [29]:
# Check if delay for more than 0 min, if yes, return 1, if no, return 0
flight_label = flight_label.withColumn(
    "Delay",
    when(col("arrival_time_difference_seconds") > 0, 1).otherwise(0)
)
#flight_label.select("published_arrival_timestamp", "original_arrival_timestamp", "Delay").show()

In [34]:
flight = flight_label.dropna().cache()

In [35]:
counts = flight.groupBy('Delay').count()

counts_pandas = counts.toPandas()

24/04/14 19:59:49 WARN org.apache.spark.storage.BlockManager: Putting block rdd_123_0 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string..
24/04/14 19:59:49 WARN org.apache.spark.storage.BlockManager: Putting block rdd_123_1 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-08-23 11:10:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string..
24/04/14 19:59:49 WARN org.apache.spark.storage.BlockManager: Block rdd_123_1 could not be removed as it was not found on disk

Py4JJavaError: An error occurred while calling o299.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 1052) (final-cluster-jy-m.us-central1-c.c.final-project-419701.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)

In [30]:
flight_label.printSchema()

root
 |-- departure_airport_code: string (nullable = true)
 |-- departure_airport_avg_altitude: double (nullable = true)
 |-- departure_airport_avg_bearing: double (nullable = true)
 |-- departure_airport_avg_knots: double (nullable = true)
 |-- departure_airport_avg_temperature: double (nullable = true)
 |-- departure_airport_min_altitude: integer (nullable = true)
 |-- departure_airport_min_bearing: integer (nullable = true)
 |-- departure_airport_min_knots: integer (nullable = true)
 |-- departure_airport_min_temperature: integer (nullable = true)
 |-- departure_airport_max_altitude: integer (nullable = true)
 |-- departure_airport_max_bearing: integer (nullable = true)
 |-- departure_airport_max_knots: integer (nullable = true)
 |-- departure_airport_max_temperature: integer (nullable = true)
 |-- arrival_airport_code: string (nullable = true)
 |-- arrival_airport_avg_altitude: double (nullable = true)
 |-- arrival_airport_avg_bearing: double (nullable = true)
 |-- arrival_airport_

In [21]:
flights_0 = flight_label.filter(flight_label['Delay'] == 0)

In [22]:
flights_1 = flight_label.filter(flight_label['Delay'] == 1)

In [23]:
flights_0.count()

24/04/14 18:52:34 WARN org.apache.spark.storage.BlockManager: Putting block rdd_123_3 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-08-20 01:05:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string..
24/04/14 18:52:34 WARN org.apache.spark.storage.BlockManager: Block rdd_123_3 could not be removed as it was not found on disk or in memory
24/04/14 18:52:34 WARN org.apache.spark.storage.BlockManager: Putting block rdd_123_0 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid date

Py4JJavaError: An error occurred while calling o231.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 20.0 failed 1 times, most recent failure: Lost task 3.0 in stage 20.0 (TID 1048) (final-cluster-jy-m.us-central1-c.c.final-project-419701.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-08-20 01:05:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.a

In [ ]:
flights_1.count()

In [32]:
label_counts = flight_label['Delay'].map(lambda x: (x[-1],1))
label_counts = label_counts.reduceByKey(lambda a, b: a + b)
label_counts_collected = label_counts.collect()

TypeError: 'Column' object is not callable

In [ ]:
label_counts_collected

In [ ]:
# Map each label to a tuple (label, 1) to prepare for counting
label_counts = rdd.map(lambda x: (x[1], 1))

# Reduce by key to count the occurrences of each label
label_counts = label_counts.reduceByKey(lambda a, b: a + b)

# Collect the results (this brings the data back to the driver node, use collect() only if the data is not too large)
label_counts_collected = label_counts.collect()

In [26]:
flight = flight_label.cache()

24/04/14 18:33:15 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


In [29]:
label_counts = flight.groupBy('Delay').count()

In [31]:
label_counts.show()

24/04/14 18:34:43 WARN org.apache.spark.storage.BlockManager: Putting block rdd_123_0 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string..
24/04/14 18:34:43 WARN org.apache.spark.storage.BlockManager: Block rdd_123_0 could not be removed as it was not found on disk or in memory
24/04/14 18:34:43 WARN org.apache.spark.storage.BlockManager: Putting block rdd_129_0 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid date

Py4JJavaError: An error occurred while calling o253.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 1046) (final-cluster-jy-m.us-central1-c.c.final-project-419701.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at 

In [27]:
label_counts = flight.map(lambda x: (x[-1],1))
label_counts = label_counts.reduceByKey(lambda a, b: a + b)
label_counts_collected = label_counts.collect()

AttributeError: 'DataFrame' object has no attribute 'map'

## Balance label

In [23]:
flight['Delay'].value_counts

DataFrame[departure_airport_code: string, departure_airport_avg_altitude: double, departure_airport_avg_bearing: double, departure_airport_avg_knots: double, departure_airport_avg_temperature: double, departure_airport_min_altitude: int, departure_airport_min_bearing: int, departure_airport_min_knots: int, departure_airport_min_temperature: int, departure_airport_max_altitude: int, departure_airport_max_bearing: int, departure_airport_max_knots: int, departure_airport_max_temperature: int, arrival_airport_code: string, arrival_airport_avg_altitude: double, arrival_airport_avg_bearing: double, arrival_airport_avg_knots: double, arrival_airport_avg_temperature: double, arrival_airport_min_altitude: int, arrival_airport_min_bearing: int, arrival_airport_min_knots: int, arrival_airport_min_temperature: int, arrival_airport_max_altitude: int, arrival_airport_max_bearing: int, arrival_airport_max_knots: int, arrival_airport_max_temperature: int, airline_code: string, flight_number: int, crea

# 3. Encoding

In [23]:
# We will focus on 10 columns 
# 1，departure_airport_code: string
# 2，arrival_airport_code: string
# 3，airline_code: string 
# 4，creator_code: string ？？
# 5, scheduled_aircraft_type: string 
# 6, icao_aircraft_type_actual: string
    # *, departure_airport: string Those two we filtered out earlier
    # *, arrival_airport: string 
# 7, aircraft_id: string 
# 8, legacy_route: string 


In [24]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline  # Corrected import here
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType
import math

columns_to_encode = [
    "departure_airport_code", "arrival_airport_code", "airline_code", 
    "creator_code", "scheduled_aircraft_type", "icao_aircraft_type_actual",
    "departure_airport", "arrival_airport", "aircraft_id", "legacy_route"
]

# Create a StringIndexer for each column and collect them in a list
indexers = [
    StringIndexer(inputCol=column, outputCol=f"{column}_Index")
    for column in columns_to_encode
]

# Use a Pipeline to apply all indexers
pipeline = Pipeline(stages=indexers)
flight_indexed = pipeline.fit(flight).transform(flight)

# Define a function to convert index to binary string
def index_to_binary(index, length):
    binary_str = bin(int(index))[2:]  # Convert to binary string
    return binary_str.zfill(length)  # Pad with zeros

# Define a function to perform binary encoding
def apply_binary_encoding(df, column):
    max_index = df.agg({f"{column}_Index": "max"}).collect()[0][0]
    num_bits = math.ceil(math.log(max_index + 1, 2))
    
    udf_convert = F.udf(lambda x: index_to_binary(x, num_bits), StringType())
    df = df.withColumn(f"{column}_binary", udf_convert(F.col(f"{column}_Index")))
    
    for bit in range(num_bits):
        bit_col_name = f"{column}_bit_{bit}"
        df = df.withColumn(bit_col_name, F.substring(F.col(f"{column}_binary"), num_bits - bit, 1).cast(IntegerType()))
    
    return df, num_bits

# Apply binary encoding to each indexed column
for column in columns_to_encode:
    flight_indexed, _ = apply_binary_encoding(flight_indexed, column)

# Optionally drop intermediate columns
columns_to_drop = [f"{column}_Index" for column in columns_to_encode] + [f"{column}_binary" for column in columns_to_encode]
flight_encoded = flight_indexed.drop(*columns_to_drop)

KeyboardInterrupt: 

In [48]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline  # Corrected import here
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType
import math

columns_to_encode = [
    "departure_airport_code", "arrival_airport_code", "airline_code", 
    "creator_code", "scheduled_aircraft_type", "icao_aircraft_type_actual", #"departure_airport", "arrival_airport", 
    "aircraft_id", "legacy_route"
]

# Create a StringIndexer for each column and collect them in a list
indexers = [
    StringIndexer(inputCol=column, outputCol=f"{column}_Index")
    for column in columns_to_encode
]

# Use a Pipeline to apply all indexers
pipeline = Pipeline(stages=indexers)
flight_indexed = pipeline.fit(flight).transform(flight)

# Define a function to convert index to binary string
def index_to_binary(index, length):
    binary_str = bin(int(index))[2:]  # Convert to binary string
    return binary_str.zfill(length)  # Pad with zeros

# Define a function to perform binary encoding
def apply_binary_encoding(df, column):
    max_index = df.agg({f"{column}_Index": "max"}).collect()[0][0]
    num_bits = math.ceil(math.log(max_index + 1, 2))
    
    udf_convert = F.udf(lambda x: index_to_binary(x, num_bits), StringType())
    df = df.withColumn(f"{column}_binary", udf_convert(F.col(f"{column}_Index")))
    
    for bit in range(num_bits):
        bit_col_name = f"{column}_bit_{bit}"
        df = df.withColumn(bit_col_name, F.substring(F.col(f"{column}_binary"), num_bits - bit, 1).cast(IntegerType()))
    
    return df, num_bits

# Apply binary encoding to each indexed column
for column in columns_to_encode:
    flight_indexed, _ = apply_binary_encoding(flight_indexed, column)

# Optionally drop intermediate columns
columns_to_drop = [f"{column}_Index" for column in columns_to_encode] + [f"{column}_binary" for column in columns_to_encode]
flight_encoded = flight_indexed.drop(*columns_to_drop)

24/04/08 03:37:07 WARN org.apache.spark.storage.BlockManager: Putting block rdd_123_0 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string..
24/04/08 03:37:07 WARN org.apache.spark.storage.BlockManager: Block rdd_123_0 could not be removed as it was not found on disk or in memory
24/04/08 03:37:07 WARN org.apache.spark.storage.BlockManager: Putting block rdd_129_0 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid date

Py4JJavaError: An error occurred while calling o549.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 1051) (final-cluster-jy-m.us-central1-c.c.final-project-419701.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apa

In [47]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline  # Corrected import here
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType
import math

columns_to_encode = [
    "departure_airport_code", "arrival_airport_code", "airline_code", 
    "creator_code", "scheduled_aircraft_type", "icao_aircraft_type_actual", #"departure_airport", "arrival_airport", 
    "aircraft_id", "legacy_route"
]

# Create a StringIndexer for each column and collect them in a list
indexers = [
    StringIndexer(inputCol=column, outputCol=f"{column}_Index", handleInvalid='skip')
    for column in columns_to_encode
]

# Use a Pipeline to apply all indexers
pipeline = Pipeline(stages=indexers)
flight_indexed = pipeline.fit(flight).transform(flight)

24/04/08 03:33:29 WARN org.apache.spark.storage.BlockManager: Putting block rdd_123_1 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-08-23 11:10:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string..
24/04/08 03:33:29 WARN org.apache.spark.storage.BlockManager: Block rdd_123_1 could not be removed as it was not found on disk or in memory
24/04/08 03:33:29 WARN org.apache.spark.storage.BlockManager: Putting block rdd_129_1 failed due to exception org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-08-23 11:10:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid date

Py4JJavaError: An error occurred while calling o520.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 20.0 failed 1 times, most recent failure: Lost task 0.0 in stage 20.0 (TID 1045) (final-cluster-jy-m.us-central1-c.c.final-project-419701.internal executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '2013-09-03 18:42:00+00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apa

In [46]:
# Define a function to convert index to binary string
def index_to_binary(index, length):
    binary_str = bin(int(index))[2:]  # Convert to binary string
    return binary_str.zfill(length)  # Pad with zeros

# Define a function to perform binary encoding
def apply_binary_encoding(df, column):
    max_index = df.agg({f"{column}_Index": "max"}).collect()[0][0]
    num_bits = math.ceil(math.log(max_index + 1, 2))
    
    udf_convert = F.udf(lambda x: index_to_binary(x, num_bits), StringType())
    df = df.withColumn(f"{column}_binary", udf_convert(F.col(f"{column}_Index")))
    
    for bit in range(num_bits):
        bit_col_name = f"{column}_bit_{bit}"
        df = df.withColumn(bit_col_name, F.substring(F.col(f"{column}_binary"), num_bits - bit, 1).cast(IntegerType()))
    
    return df, num_bits

# Apply binary encoding to each indexed column
for column in columns_to_encode:
    flight_indexed, _ = apply_binary_encoding(flight_indexed, column)

Py4JJavaError: An error occurred while calling o1803.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 419.0 failed 1 times, most recent failure: Lost task 0.0 in stage 419.0 (TID 10072, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2083)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1657)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2431)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2355)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2213)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1669)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2431)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2355)
	at java.io.ObjectInputStream.readOrdinary

In [ ]:
# Optionally drop intermediate columns
columns_to_drop = [f"{column}_Index" for column in columns_to_encode] + [f"{column}_binary" for column in columns_to_encode]
flight_encoded = flight_indexed.drop(*columns_to_drop)